This package is to run data modelling/forecast predictions based on historical data.

The sqlite database keeps a record of what stock data is on hand, when it begins, and when it ends.

When data is requested from the database, if it is not available in the database, execute a request to the API (and update the ticker table as well as the metadata table in the database)

In [1]:
import datetime as dt
import yfinance as yf
import sqlite3
import pandas as pd
import time 
from db_utilities import query, list_all_tables, create_metadata_table, create_ticker_table, drop_ticker_table, read_table, update_metadata, create_metadata, initial_data_pull
from stock_functions import get_close_price, get_ohlc
# populate your universe of stocks
data = pd.read_csv('data/constituents.csv')
all_tickers = sorted(list(data['Symbol']))

In [2]:
drop_ticker_table('stock_metadata')
for x in list_all_tables():
    if x in all_tickers:
        drop_ticker_table(x)
create_metadata_table()
print(len(list_all_tables()))

dropped table stock_metadata
2


In [3]:
con = sqlite3.connect('stocks.db')
res = con.execute('select * from stock_metadata')
# print(res.description)
res.fetchall()

[]

In [4]:
for x in sorted(all_tickers):
    print(x)
    initial_data_pull(x)
    time.sleep(2)


c:\Users\polan\Desktop\GitHub Repos\A New Project\myenv\Lib\site-packages\yfinance\utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
c:\Users\polan\Desktop\GitHub Repos\A New Project\myenv\Lib\site-packages\yfinance\utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
c:\Users\polan\Desktop\GitHub Repos\A New Project\myenv\Lib\site-packages\yfinance\utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
c:\Users\polan\Desktop\GitHub Repos\A New Project\myenv\Lib\site-packages\yfinance\utils.py:775: 

In [9]:
get_ohlc('ABC')

ABC: No timezone found, symbol may be delisted


,Date,Open,High,Low,Close,Adj Close,Volume


In [17]:
fails = []
summary = dict()
for x in all_tickers:
    summary[x] = query(f'select count(*) as rowcount, min(date) as max_date, max(date) as max_date from "{x}"')
    if summary[x]['rowcount'][0] < 2000:
        fails.append(x)

In [19]:
len(summary)

505

In [18]:
len(fails)

42